In [0]:
CREATE OR REPLACE TABLE inceptez_catalog.inputdb.tblcusts USING DELTA AS
SELECT
  _c0 AS custid,
  _c1 AS fname,
  _c2 AS lname,
  _c3 AS age,
  _c4 AS prof
FROM
  csv.`dbfs:/Volumes/inceptez_catalog/inputdb/customerdata/custs` WITH (inferSchema = "true");

select
  *
from
  inceptez_catalog.inputdb.tblcusts
limit 10;

--get the list of customers whose profession is Pilot and located in California
select
  c.custid,
  c.fname,
  c.lname,
  t.amount,
  t.state,
  t.city
from
  inceptez_catalog.inputdb.tblcusts c
    inner join inceptez_catalog.inputdb.tbltxns t
      on c.custid = t.custid
where
  prof = 'Pilot'
  and state = 'California';

select distinct
  c.custid,
  c.fname,
  c.lname
from
  inceptez_catalog.inputdb.tblcusts c inner join inceptez_catalog.inputdb.tbltxns t
where
  prof = 'Pilot'
  and state = 'California';

--Derived Query
select
  *
from
  (
    select
      *
    from
      inceptez_catalog.inputdb.tbltxns
    where
      state = 'California'
  ) t
    inner join (
      select
        *
      from
        inceptez_catalog.inputdb.tblcusts
      where
        prof = 'Pilot'
    ) c
      on c.custid = t.custid;

--sum(sales) amount by state > 50000
select
  state,
  round(total_sales, 2)
from
  (
    select
      state,
      sum(amount) as total_sales
    from
      inceptez_catalog.inputdb.tbltxns
    group by
      state
  ) s
where
  total_sales > 50000;

select
  custid,
  count(txnid)
from
  inceptez_catalog.inputdb.tbltxns
group by
  custid
having
  count(txnid) > 10;

--derived
select
  *
from
  (
    select
      custid,
      count(txnid) count
    from
      inceptez_catalog.inputdb.tbltxns
    group by
      custid
  ) cust_cnt
where
  count > 10;



--CTE
with cust_cnt as (
  select
    custid,
    count(txnid) count
  from
    inceptez_catalog.inputdb.tbltxns
  group by
    custid
)
select
  *
from
  cust_cnt
where
  count > 10;




with txn_data as (
  select
    *
  from
    inceptez_catalog.inputdb.tbltxns
  where
    state = 'California'
),
cust_data as (
  select
    *
  from
    inceptez_catalog.inputdb.tblcusts
  where
    prof = 'Pilot'
)
select
  c.custid,
  c.fname,
  c.lname,
  t.amount,
  t.state,
  t.city
from
  txn_data t
    inner join cust_data c
      on c.custid = t.custid;

-- total sales by state and it should be greater that the average total sales of all states

WITH total_sales AS (
  SELECT
    state,
    SUM(amount) AS total_amount
  FROM
    inceptez_catalog.inputdb.tbltxns
  GROUP BY
    state
),
avg_sales AS (
  SELECT
    AVG(total_amount) AS avg_state_sales
  FROM
    total_sales
)
SELECT
  *
FROM
  total_sales,
  avg_sales
WHERE
  total_sales.total_amount > avg_sales.avg_state_sales;



  